In [22]:
import pandas as pd

df = pd.read_csv("../src/state/exp.csv", index_col=0).reset_index(drop=True)
df.head()

,stopName,stopDesc,zoneId,zoneName,stopLat,stopLon,coord,time_sec,color
0,Św. Wojciech,Święty Wojciech,1.0,Gdańsk,54.28478,18.63808,"54.28478000000001, 18.63808",3660,black
1,Olimpijska,Olimpijska,1.0,Gdańsk,54.30558,18.58112,"54.305580000000006, 18.58112",3997,black
2,Maćkowy,Maćkowy,1.0,Gdańsk,54.30689,18.61525,"54.30689, 18.61525",3780,black
3,Przegalina,Przegalina,1.0,Gdańsk,54.31122,18.91707,"54.31122, 18.91707",5160,black
4,Kampinoska,Kampinoska,1.0,Gdańsk,54.31611,18.60301,"54.31611, 18.60301",3360,red


In [2]:
Lat_max, Lat_min = df.stopLat.max(), df.stopLat.min()
Lon_max, Lon_min = df.stopLon.max(), df.stopLon.min()

In [3]:
map_3city_points = df[['stopLon', 'stopLat']].values

In [4]:
import numpy as np
from scipy.spatial import Voronoi

stops = df.coord.values
ids = np.array(map_3city_points)
vor = Voronoi(ids)
polygons = {}
for id, region_index in enumerate(vor.point_region):
    points = []
    for vertex_index in vor.regions[region_index]:
        if vertex_index != -1:  # the library uses this for infinity
            points.append(list(vor.vertices[vertex_index]))
    points.append(points[0])
    polygons[id]=points

In [5]:
stops_indexes = [row['coord'] for index, row in df.iterrows()]

In [6]:
data_geoJson = [{index: coordinates} for index, coordinates in zip(stops_indexes, polygons.values())]

In [94]:
boundary = np.array([[54.354509, 18.948932],
                     [54.436059, 18.606983],
                     [54.596288, 18.558918],
                     [54.594697, 18.341938],
                     [54.388688, 18.411136],
                     [54.264611, 18.539508],
                     [54.297227, 18.929380]])

In [90]:
geoJson_3city = {"type":"FeatureCollection","features":[
                    {"type":"Feature",
                     "id":list(zone.keys())[0],
                     "properties":{"name":"3city"},
                     "geometry":{"type":"Polygon",
                                 "coordinates": list(zone.values())}} for zone in data_filtered_geoJson]}

In [91]:
import json
with open('../src/state/geoJson_3city_filtered.json', 'w') as fp:
    json.dump(geoJson_3city, fp)

In [93]:
import folium
import os

# city_geo = os.path.join('../src/state/', 'geoJson_3city_filtered.json')
city_geo = os.path.join('../src/state/', 'geoJson_3city.json')

time_data = pd.read_csv("../src/state/exp.csv", index_col=0).reset_index(drop=True).\
                                                             rename(columns={'coord':'id'})
time_data['time_min'] = time_data['time_sec'] / 60

m = folium.Map(location=[54.441564, 18.562754], zoom_start = 10)

folium.Choropleth(
    geo_data=city_geo,
    name='choropleth',
    data=time_data,
    columns=['id', 'time_min'],
    key_on='feature.id',
    line_weight=2,
    fill_color='BuPu',
    fill_opacity=0.4,
    line_opacity=0.2,
    legend_name='Time (min)',
    bins=9
).add_to(m)

df['Lat'] = df['stopLat'].astype(float)
df['Lon'] = df['stopLon'].astype(float)

heat_df = df[['Lat', 'Lon']]
heat_data = [[row['Lat'], row['Lon']] for index, row in heat_df.iterrows()]

for index, row in heat_df.iterrows():
    folium.Circle(location=[row['Lat'], row['Lon']],
                  radius=10,
                  color=None,
                  fill=True,
                  fill_color="black",
                  fill_opacity=1).add_to(m)

m#.save("folium_choropleth_3city.html")

## polygons test

In [86]:
zone0 = polygons[0]

In [83]:
zone0

[[18.666614531584777, 54.293085947922734],
 [18.669398764121773, 54.28887536310553],
 [18.612974768914558, 54.27586174804933],
 [18.616674586249673, 54.28219605735879],
 [18.66523379200332, 54.29481448896145],
 [18.666614531584777, 54.293085947922734]]

In [91]:
test_point0 = [df.loc[0,:].stopLat, df.loc[0,:].stopLon]
test_point0

[54.28478000000001, 18.63808]

In [92]:
zone100 = polygons[100]
zone100

[[18.776527057301692, 54.15664022430251],
 [18.860895296620388, 53.571363695110655],
 [18.253387076257052, 53.5136813578189],
 [18.58676068162788, 54.24089461141939],
 [18.612974768914558, 54.27586174804933],
 [18.669398764121773, 54.28887536310553],
 [18.776527057301692, 54.15664022430251]]

In [93]:
test_point100 = [df.loc[100,:].stopLat, df.loc[100,:].stopLon]
test_point100

[54.27884, 18.63945]

In [114]:
from folium import plugins
from folium.plugins import HeatMap
import folium

map_3city = folium.Map(location=[54.441564, 18.562754], zoom_start = 10) 

heat_data = [zone0, zone100]

folium.Circle(location=test_point0,
              radius=150,
              color=None,
              fill=True,
              fill_color="red",
              fill_opacity=1).add_to(map_3city)

folium.Circle(location=test_point100,
              radius=150,
              color=None,
              fill=True,
              fill_color="black",
              fill_opacity=1).add_to(map_3city)


for point in heat_data[0]:
    print(point)
    folium.Circle(location=[point[1], point[0]],
                  radius=150,
                  color=None,
                  fill=True,
                  fill_color="green",
                  fill_opacity=1).add_to(map_3city)
    
for point in heat_data[1]:
    folium.Circle(location=[point[1], point[0]],
                  radius=150,
                  color=None,
                  fill=True,
                  fill_color="blue",
                  fill_opacity=0.2).add_to(map_3city)

    

map_3city

[18.666614531584777, 54.293085947922734]
[18.669398764121773, 54.28887536310553]
[18.612974768914558, 54.27586174804933]
[18.616674586249673, 54.28219605735879]
[18.66523379200332, 54.29481448896145]
[18.666614531584777, 54.293085947922734]
